<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/dev.Andrew/GTFS_ABQ_load_pickle_set_params_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ABC note 250726-1715: I have some concerns about the anomaly placement, but I also haven't worked with a multi-day dataset previously, so I need to do some more testing.


#Load the Libraries

In [ ]:
#Required Libraries
import pandas as pd
import numpy as np

# # Install if needed
# !pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

from datetime import datetime
import zipfile

from shapely.geometry import Point, LineString
from geopy.distance import geodesic

from folium import Map, FeatureGroup, CircleMarker, PolyLine, Marker, Icon, LayerControl
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import folium

In [ ]:
# Select Parameters for Anomaly Detection

# --- Use meters for user input ---
jump_thresh_m_widget = widgets.IntSlider(
    value=500,
    min=50,
    max=2000,
    step=50,
    description='Jump Distance',
    layout=widgets.Layout(width='60%')
)

# Other anomaly widgets
disappear_thresh_widget = widgets.IntSlider(value=300, min=60, max=1800, step=30, description='Time Gap')
min_jump_widget = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Min Jumps')
speed_thresh_widget = widgets.IntSlider(value=70, min=10, max=100, step=5, description='Speed')
reversal_thresh_widget = widgets.IntSlider(value=120, min=60, max=180, step=5, description='Heading Δ°')

# --- Stuck vehicle detection ---
stuck_speed_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Stuck Speed',
    layout=widgets.Layout(width='60%')
)

stuck_window_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Window Size',
    layout=widgets.Layout(width='60%')
)

# --- Repeated point tolerance ---
repeat_tolerance_widget = widgets.FloatLogSlider(
    value=1e-5,
    base=10,
    min=-7,  # 1e-7
    max=-3,  # 1e-3
    step=0.1,
    description='Repeat Tolerance',
    layout=widgets.Layout(width='60%')
)

# --- Early appearance margin ---
early_margin_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=600,
    step=10,
    description='Early Margin',
    layout=widgets.Layout(width='60%')
)

# --- Off-route buffer ---
offroute_buffer_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description='Off-Route Buffer',
    layout=widgets.Layout(width='60%')
)

# Labels
jump_label = widgets.Label(value="(meters)")
disappear_label = widgets.Label(value="(seconds)")
min_jump_label = widgets.Label(value="(count)")
speed_label = widgets.Label(value="(mph)")
reversal_label = widgets.Label(value="(degrees of reversal)")
stuck_speed_label = widgets.Label(value="(mph)")
stuck_window_label = widgets.Label(value="(frames in rolling window)")
repeat_tolerance_label = widgets.Label(value="(decimal degrees)")
early_margin_label = widgets.Label(value="(seconds)")
offroute_buffer_label = widgets.Label(value="(meters)")

# Assemble UI layout
slider_widgets = widgets.VBox([
    widgets.HBox([jump_thresh_m_widget, jump_label]),
    widgets.HBox([disappear_thresh_widget, disappear_label]),
    widgets.HBox([min_jump_widget, min_jump_label]),
    widgets.HBox([speed_thresh_widget, speed_label]),
    widgets.HBox([reversal_thresh_widget, reversal_label]),
    widgets.HBox([stuck_speed_widget, stuck_speed_label]),
    widgets.HBox([stuck_window_widget, stuck_window_label]),
    widgets.HBox([repeat_tolerance_widget, repeat_tolerance_label]),
    widgets.HBox([early_margin_widget, early_margin_label]),
    widgets.HBox([offroute_buffer_widget, offroute_buffer_label]),
])

# Button and save logic
submit_button = widgets.Button(description="Save Parameters", button_style='primary')
anomaly_params = {}

def save_params(b):
    clear_output(wait=True)
    display(slider_widgets, submit_button)

    global anomaly_params
    jump_m = jump_thresh_m_widget.value
    jump_deg = jump_m / 111000  # Convert meters → degrees

    anomaly_params = {
        "JUMP_DISTANCE_THRESHOLD": jump_deg,
        "JUMP_DISTANCE_METERS": jump_m,
        "DISAPPEARANCE_TIME_THRESHOLD": disappear_thresh_widget.value,
        "MIN_JUMP_COUNT_PER_VEHICLE": min_jump_widget.value,
        "SPEED_LIMIT_MPH": speed_thresh_widget.value,
        "REVERSAL_HEADING_THRESHOLD": reversal_thresh_widget.value,
        "STUCK_SPEED_MPH": stuck_speed_widget.value,
        "STUCK_WINDOW_SIZE": stuck_window_widget.value,
        "REPEATED_COORD_TOLERANCE": repeat_tolerance_widget.value,
        "EARLY_APPEARANCE_MARGIN_SEC": early_margin_widget.value,
        "OFF_ROUTE_BUFFER_M": offroute_buffer_widget.value,
    }

    print("✅ Anomaly detection parameters set:")
    for k, v in anomaly_params.items():
        print(f"  {k}: {v}")

submit_button.on_click(save_params)

# Display interface
display(slider_widgets, submit_button)


Button(button_style='primary', description='Save Parameters', style=ButtonStyle())

✅ Anomaly detection parameters set:
  JUMP_DISTANCE_THRESHOLD: 0.0045045045045045045
  JUMP_DISTANCE_METERS: 500
  DISAPPEARANCE_TIME_THRESHOLD: 300
  MIN_JUMP_COUNT_PER_VEHICLE: 2
  SPEED_LIMIT_MPH: 70
  REVERSAL_HEADING_THRESHOLD: 120
  STUCK_SPEED_MPH: 1.0
  STUCK_WINDOW_SIZE: 4
  REPEATED_COORD_TOLERANCE: 1e-05
  EARLY_APPEARANCE_MARGIN_SEC: 30
  OFF_ROUTE_BUFFER_M: 50


#LOAD PICKLE

In [ ]:
import pickle

with open("captures.p", "rb") as f:
    captured_data = pickle.load(f)


##Data Validation

In [ ]:
cd_df = captured_data
cd_df.size

13158480

In [ ]:
# Display full list of column names
print("Number of columns:", len(cd_df.columns))
print("Column names:\n", cd_df.columns.tolist())

Number of columns: 12
Column names:
 ['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude', 'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time']


In [ ]:
cd_df.head()

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
0,1,2025-07-22T11:17:02.268350,610,35.147320,-106.639650,186.0,9.65604,10,0,0,4th @ Guadalupe,11:32:57
1,1,2025-07-22T11:17:02.268350,358,35.106341,-106.693167,96.0,64.00000,11,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
2,1,2025-07-22T11:17:02.268350,359,35.152506,-106.694041,92.0,0.00000,157,0,0,NW Transit Center (Bay C),05:31:00
3,1,2025-07-22T11:17:02.268350,383,35.058513,-106.569910,17.0,0.00000,16,601256,2801,Gibson @ San Pedro,15:18:45
4,1,2025-07-22T11:17:02.268350,627,35.106680,-106.550930,178.0,0.00000,31,0,0,Wyoming @ Northeastern,13:03:07


In [ ]:
# Convert timestamp to datetime
cd_df['timestamp_collected'] = pd.to_datetime(cd_df['timestamp_collected'], errors='coerce')

# 1. Shape and Column Overview
print("Shape:", cd_df.shape)
print("Columns:", cd_df.columns.tolist())

# 2. Null/Missing Value Counts
print("\nMissing Values:\n", cd_df.isnull().sum())

# 3. Duplicate Detection
duplicates = cd_df.duplicated().sum()
print("\nDuplicate rows:", duplicates)

# 4. Timestamp Range
print("\nTimestamp Range:")
print("Min:", cd_df['timestamp_collected'].min())
print("Max:", cd_df['timestamp_collected'].max())
print("Duration:", cd_df['timestamp_collected'].max() - cd_df['timestamp_collected'].min())

# 5. Latitude/Longitude Range
print("\nLatitude Range:", cd_df['latitude'].min(), "to", cd_df['latitude'].max())
print("Longitude Range:", cd_df['longitude'].min(), "to", cd_df['longitude'].max())

# 6. Speed Summary
print("\nSpeed Summary:\n", cd_df['speed_mph'].describe())


Shape: (1096540, 12)
Columns: ['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude', 'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time']

Missing Values:
 snapshot_id             0
timestamp_collected     0
vehicle_id              0
latitude                0
longitude               0
heading                 0
speed_mph               0
route_short_name        0
trip_id                 0
next_stop_id            0
next_stop_name          0
next_stop_sched_time    0
dtype: int64

Duplicate rows: 0

Timestamp Range:
Min: 2025-07-22 11:17:02.268350
Max: 2025-07-24 01:13:30.399545
Duration: 1 days 13:56:28.131195

Latitude Range: 0.0 to 35.4133163
Longitude Range: -106.79815 to 0.0

Speed Summary:
 count    1.096540e+06
mean     4.230205e+00
std      1.108069e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.000000e+01
Name: speed_mph, dtype: float64


##✅ Overall Status
Your dataset appears valid and cleanly structured for analysis. You can safely proceed with anomaly detection, with only minor issues to address or monitor.

| Category            | Result                               | Notes                                                                          |
| ------------------- | ------------------------------------ | ------------------------------------------------------------------------------ |
| **Shape**           | (1,096,540 rows, 12 columns)         | Substantial volume, well-sized for analysis                                    |
| **Missing Values**  | 0 missing in any column              | ✅ Excellent data integrity                                                     |
| **Duplicates**      | 0 duplicate rows                     | ✅ Clean snapshot logic                                                         |
| **Timestamp Range** | \~1.6 days                           | ⚠️ Slightly under 2 days; may be due to gaps or partial logging                |
| **Latitude Range**  | `0.0` to `35.41`                     | ⚠️ `0.0` indicates corrupted or failed GPS points — needs filtering            |
| **Longitude Range** | `-106.79` to `0.0`                   | ⚠️ Same issue — longitudes of `0.0` are invalid (off the west coast of Africa) |
| **Speed (mph)**     | Mean: `4.2`, Max: `80`, 75% at `0.0` | ⚠️ Excessive zero speeds → vehicles idle or unreported? Worth mapping          |
| **Outliers**        | Max speed = `80 mph`                 | 🔍 Borderline — worth flagging for manual inspection                           |


In [ ]:
#Some preliminary prep for analysis

# Remove invalid GPS coordinates
cd_df_valid = cd_df[(cd_df['latitude'] > 30) & (cd_df['longitude'] < -100)]

# Optional: filter out idle snapshots (speed = 0) if focusing on movement
cd_df_moving = cd_df_valid[cd_df_valid['speed_mph'] > 0]


#Convert code for snapshot to use Pickle

In [ ]:
def filter_cd_df(df, start_time=None, end_time=None, routes=None, vehicles=None, bounds=None):
    df_filtered = df.copy()

    if start_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] >= pd.to_datetime(start_time)]
    if end_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] <= pd.to_datetime(end_time)]
    if routes:
        df_filtered = df_filtered[df_filtered['route_short_name'].isin(routes)]
    if vehicles:
        df_filtered = df_filtered[df_filtered['vehicle_id'].isin(vehicles)]
    if bounds:
        lat_min, lat_max, lon_min, lon_max = bounds
        df_filtered = df_filtered[
            (df_filtered['latitude'] >= lat_min) & (df_filtered['latitude'] <= lat_max) &
            (df_filtered['longitude'] >= lon_min) & (df_filtered['longitude'] <= lon_max)
        ]

    return df_filtered


In [ ]:


# --- Widgets ---
route_widget = widgets.SelectMultiple(
    options=sorted(cd_df['route_short_name'].unique()),
    description='Routes',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

vehicle_widget = widgets.SelectMultiple(
    options=sorted(cd_df['vehicle_id'].unique()),
    description='Vehicles',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

start_widget = widgets.DatePicker(
    description='Start Date',
    value=cd_df['timestamp_collected'].min().date()
)

end_widget = widgets.DatePicker(
    description='End Date',
    value=cd_df['timestamp_collected'].max().date()
)

hour_slider = widgets.IntRangeSlider(
    value=[0, 23],
    min=0,
    max=23,
    step=1,
    description='Hour Range',
    layout=widgets.Layout(width='60%')
)

filter_button = widgets.Button(description='Apply Filters', button_style='primary')

# --- Global to store result ---
cd_df_filtered = pd.DataFrame()

# --- Filtering callback ---
def apply_filters(b):
    global cd_df_filtered
    clear_output(wait=True)
    display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)

    df = cd_df.copy()

    # Apply filters
    if route_widget.value:
        df = df[df['route_short_name'].isin(route_widget.value)]
    if vehicle_widget.value:
        df = df[df['vehicle_id'].isin(vehicle_widget.value)]

    start_dt = pd.to_datetime(start_widget.value)
    end_dt = pd.to_datetime(end_widget.value) + pd.Timedelta(days=1)
    df = df[(df['timestamp_collected'] >= start_dt) &
            (df['timestamp_collected'] < end_dt)]

    hr_start, hr_end = hour_slider.value
    df = df[(df['timestamp_collected'].dt.hour >= hr_start) &
            (df['timestamp_collected'].dt.hour <= hr_end)]

    # Store result globally
    cd_df_filtered = df

    print(f"✅ Filtered {len(df)} rows.")
    display(df.head(10))

filter_button.on_click(apply_filters)

# --- Display UI ---
display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)


SelectMultiple(description='Routes', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

SelectMultiple(description='Vehicles', layout=Layout(width='50%'), options=(np.int64(351), np.int64(352), np.i…

DatePicker(value=datetime.date(2025, 7, 22), description='Start Date')

DatePicker(value=datetime.date(2025, 7, 24), description='End Date')

IntRangeSlider(value=(0, 23), description='Hour Range', layout=Layout(width='60%'), max=23)

Button(button_style='primary', description='Apply Filters', style=ButtonStyle())

✅ Filtered 1096540 rows.


,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
0,1,2025-07-22 11:17:02.268350,610,35.147320,-106.639650,186.0,9.65604,10,0,0,4th @ Guadalupe,11:32:57
1,1,2025-07-22 11:17:02.268350,358,35.106341,-106.693167,96.0,64.00000,11,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
2,1,2025-07-22 11:17:02.268350,359,35.152506,-106.694041,92.0,0.00000,157,0,0,NW Transit Center (Bay C),05:31:00
3,1,2025-07-22 11:17:02.268350,383,35.058513,-106.569910,17.0,0.00000,16,601256,2801,Gibson @ San Pedro,15:18:45
4,1,2025-07-22 11:17:02.268350,627,35.106680,-106.550930,178.0,0.00000,31,0,0,Wyoming @ Northeastern,13:03:07
5,1,2025-07-22 11:17:02.268350,617,35.105550,-106.667950,76.0,64.00000,5,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
6,1,2025-07-22 11:17:02.268350,961,35.089586,-106.732609,294.0,0.00000,54,0,0,A.T.C. (Bay L),06:21:00
7,1,2025-07-22 11:17:02.268350,368,35.105217,-106.641143,96.0,64.00000,66,0,0,Wenonah @ Tramway,05:29:00
8,1,2025-07-22 11:17:02.268350,1914,35.089850,-106.732920,88.0,0.00000,766,0,0,CUTC Bay B,05:21:00
9,1,2025-07-22 11:17:02.268350,1921,35.078768,-106.521377,115.0,56.00000,766,0,0,Wenonah @ Tramway (ART),05:18:00


In [ ]:
#Sanity check following filtering
cd_df_filtered.describe()
# or
cd_df_filtered['vehicle_id'].value_counts()


,count
vehicle_id,
454,3228
610,3228
358,3228
2419,3228
2416,3228
...,...
961,3228
617,3228
627,3228


In [ ]:
#Pickle replacement
df_new = cd_df_filtered

#Old code:
#✅ Step 1a: Load and Inspect the New Snapshot
# new_snapshot_path = "/content/cabq_gtfs_snapshots_20250722_1415.csv"
# df_new = pd.read_csv(new_snapshot_path)
# df_new.info()
# df_new.head(3)

# #Also print the columns:
# print(df_new.columns.tolist())


In [ ]:
#✅ Step 2a: Trip ID Validity
df_new["trip_id"] = df_new["trip_id"].astype(str)
invalid_trip_ids = df_new["trip_id"].isin(["0", "Undetermined", "nan", "", "None"]).sum()
total_rows = len(df_new)

print(f"Total rows: {total_rows}")
print(f"Invalid trip_ids: {invalid_trip_ids}")
print(f"Percent valid: {100 * (total_rows - invalid_trip_ids) / total_rows:.2f}%")


Total rows: 1096540
Invalid trip_ids: 925184
Percent valid: 15.63%


#Load the Static data

In [ ]:
#✅ Step 1b: Reload Static GTFS and Normalize It

# Adjust if needed — make sure this is the static feed aligned with 2025-07-22
gtfs_zip_path = "/content/google_transit.zip"

with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

trips = pd.read_csv("/content/gtfs_static/trips.txt", dtype=str)
routes = pd.read_csv("/content/gtfs_static/routes.txt", dtype=str)


In [ ]:
#✅ Step 2b: Filter invalid trip_ids
invalid_trip_ids = ["0", "Undetermined", "nan", "", "None"]
df_clean = df_new[~df_new["trip_id"].isin(invalid_trip_ids)].copy()


In [ ]:
#✅ Step 3b: Merge with trips.txt to Get route_id
df_with_trips = df_clean.merge(trips, on="trip_id", how="left")


In [ ]:
#✅ Step 4b: Merge with routes.txt to Get Descriptive Info
df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# Ensure consistent types
trips["shape_id"] = trips["shape_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

df_full = df_with_trips.merge(routes, on="route_id", how="left")


In [ ]:
#✅ Step 5b: Load shapes.txt
shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)

# Merge trips and routes to link shape_id to route_short_name
shape_route_map = (
    trips.merge(routes, on="route_id", how="left")
         .dropna(subset=["route_short_name"])
         .drop_duplicates(subset=["shape_id"])
         .set_index("shape_id")[["route_id", "route_short_name"]]
)

In [ ]:
#✅ Step 6b: Create a data_quality Flag
def classify_row(row):
    if row["trip_id"] in invalid_trip_ids:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]):
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

df_full["data_quality"] = df_full.apply(classify_row, axis=1)
print(df_full["data_quality"].value_counts())


data_quality
valid    171356
Name: count, dtype: int64


In [ ]:
#Load the trips, routes, shapes from Static data
with zipfile.ZipFile(gtfs_zip_path, 'r') as z:
    trips_df = pd.read_csv(z.open("trips.txt"))
    routes_df = pd.read_csv(z.open("routes.txt"))
    # Load GTFS shapes.txt into a DataFrame
    shapes_df = pd.read_csv(z.open("shapes.txt"))

#Clean the RT data

In [ ]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 87


#Anomly Detection

🔹 1. Detect Jumps and Gaps

Already implemented, but here’s the modular form:

In [ ]:
# 🔹 1. Detect Jumps and Gaps
def detect_jumps_and_gaps(df, params):
    """
    Detects jump or gap anomalies based on distance (meters) and time (seconds)
    thresholds pulled from the anomaly_params dictionary.

    Parameters:
        df (pd.DataFrame): Filtered DataFrame for a single vehicle
        params (dict): Anomaly detection thresholds from widget interface

    Returns:
        List[dict]: List of detected jump or gap anomalies
    """
    distance_threshold = params["JUMP_DISTANCE_METERS"]           # in meters
    time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # in seconds

    anomalies = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        if time_diff > time_threshold or distance > distance_threshold:
            anomalies.append({
                "vehicle_id": row_curr["vehicle_id"],
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": distance,
                "is_gap": time_diff > time_threshold,
                "is_jump": distance > distance_threshold,
                "anomaly_type": "jump_or_gap"
            })

    return anomalies
#Note: changed row["timestamp_collected"] to row["timestamp"] throughout, assuming that we've already converted timestamp_collected to UTC and assigned it to a new "timestamp" column at the start of the anomaly pipeline (which the existing pipeline does). This avoids inconsistency.

In [ ]:
#2. Detect Stuck Vehicles
def detect_stuck_vehicle(df, params):
    """
    Detects stuck vehicles using a rolling window of speed and position stability.

    Parameters:
        df (pd.DataFrame): Single-vehicle or filtered DataFrame.
        params (dict): Widget-defined anomaly thresholds.

    Returns:
        pd.DataFrame: Rows flagged as 'stuck_vehicle'.
    """
    speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # Default: 1 mph
    window = params.get("STUCK_WINDOW_SIZE", 4)         # Default: 4-frame window

    stuck_flags = (
        (df["speed_mph"].rolling(window).mean() < speed_thresh) &
        (df["latitude"].diff().abs().rolling(window).mean() < 0.0001) &
        (df["longitude"].diff().abs().rolling(window).mean() < 0.0001)
    )

    return df[stuck_flags.fillna(False)].assign(anomaly_type="stuck_vehicle")


In [ ]:
# 🔹 3. Detect Impossible Speeds
def detect_impossible_speeds(df, params):
    """
    Detects movement segments where computed speed exceeds threshold.

    Parameters:
        df (pd.DataFrame): Filtered or per-vehicle data
        params (dict): Anomaly detection parameters from UI

    Returns:
        pd.DataFrame: Rows with flagged impossible-speed anomalies
    """
    # Convert mph to kph for geodesic-based calc
    speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934

    records = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
        if time_diff == 0:
            continue

        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        speed_kph = (distance / time_diff) * 3.6  # m/s → km/h

        if speed_kph > speed_limit_kph:
            records.append({
                "vehicle_id": row_curr["vehicle_id"],
                "timestamp_curr": row_curr["timestamp"],
                "computed_speed_kph": speed_kph,
                "distance_m": distance,
                "anomaly_type": "impossible_speed"
            })

    return pd.DataFrame(records)


In [ ]:
# 🔹 4. Detect Backtracking (Heading Reversal)
def detect_backtracking(df, params):
    """
    Detects heading reversals suggesting backtracking behavior.

    Parameters:
        df (pd.DataFrame): Filtered or per-vehicle data
        params (dict): Anomaly detection parameters from UI

    Returns:
        pd.DataFrame: Rows where heading reversed beyond threshold
    """
    reversal_thresh = params["REVERSAL_HEADING_THRESHOLD"]

    if "heading" not in df.columns:
        return pd.DataFrame()

    heading_diff = df["heading"].diff().abs()
    backtrack_flags = heading_diff.between(reversal_thresh, 200)

    return df[backtrack_flags.fillna(False)].assign(anomaly_type="backtracking")


In [ ]:
# 🔹 5. Detect Repeated Points
def detect_repeated_points(df, params):
    """
    Detects repeated GPS coordinates (vehicle not moving).

    Parameters:
        df (pd.DataFrame): Filtered or per-vehicle snapshot data
        params (dict): Anomaly detection parameters from UI

    Returns:
        pd.DataFrame: Rows flagged as having repeated lat/lon values
    """
    tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))

    if "latitude" not in df.columns or "longitude" not in df.columns:
        return pd.DataFrame()

    repeated = (
        (df["latitude"].diff().abs() < tolerance) &
        (df["longitude"].diff().abs() < tolerance)
    )

    return df[repeated.fillna(False)].assign(anomaly_type="repeated_points")


In [ ]:
# 🔹 6. Detect Disappearance Without Return
def detect_disappeared(df, snapshot_end_time, params):
    """
    Detects vehicles that have not reappeared by the end of the snapshot period.

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_end_time (datetime): End of data collection
        params (dict): Anomaly detection thresholds from widget

    Returns:
        pd.DataFrame: Single-row disappearance anomaly, or empty DataFrame
    """
    if df.empty:
        return pd.DataFrame()

    last_seen = df["timestamp"].max()
    time_gap_sec = (snapshot_end_time - last_seen).total_seconds()
    disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]

    if time_gap_sec > disappearance_thresh:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "last_seen": last_seen,
            "anomaly_type": "disappearance"
        }])

    return pd.DataFrame()


In [ ]:
# 🔹 7. Detect Early Appearance
def detect_early_appearance(df, snapshot_start_time, params):
    """
    Detects vehicles that appear too early (likely pre-start ghost data).

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_start_time (datetime): Start of capture window
        params (dict): Anomaly detection thresholds from widget

    Returns:
        pd.DataFrame: Single-row anomaly, or empty DataFrame
    """
    margin_seconds = params.get("EARLY_APPEARANCE_MARGIN_SEC", 30)
    first_seen = df["timestamp"].min()
    delta = (first_seen - snapshot_start_time).total_seconds()

    if delta < margin_seconds:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "first_seen": first_seen,
            "anomaly_type": "early_appearance"
        }])
    return pd.DataFrame()


In [ ]:
# 🔹 8. Detect Off-Route Movement
def detect_off_route(df_vehicle, shape_lines, params):
    """
    Flags GPS points that are farther than OFF_ROUTE_BUFFER_M from the expected route shape.

    Parameters:
        df_vehicle (pd.DataFrame): All points for a single vehicle
        shape_lines (dict): Dictionary of LineStrings per shape_id
        params (dict): Thresholds including OFF_ROUTE_BUFFER_M

    Returns:
        pd.DataFrame: Off-route GPS points
    """
    buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
    records = []

    for _, row in df_vehicle.iterrows():
        shape_id = str(row.get("shape_id"))
        if shape_id not in shape_lines:
            continue  # shape not available

        route_line = shape_lines[shape_id]
        vehicle_point = Point(row["longitude"], row["latitude"])

        # Project point onto shape line and compute geodesic distance
        closest_point = route_line.interpolate(route_line.project(vehicle_point))
        dist_m = geodesic(
            (row["latitude"], row["longitude"]),
            (closest_point.y, closest_point.x)
        ).meters

        if dist_m > buffer_m:
            records.append({
                "vehicle_id": row["vehicle_id"],
                "timestamp": row["timestamp"],
                "route_short_name": row.get("route_short_name"),
                "distance_from_route_m": dist_m,
                "latitude": row["latitude"],
                "longitude": row["longitude"],
                "shape_id": shape_id,
                "anomaly_type": "off_route"
            })

    return pd.DataFrame(records)


In [ ]:
# Step 1: Normalize timestamp and rebuild vehicle groups
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

# Step 2: Detect each anomaly type using widget-supplied parameters
jumpgap_records = []
stuck_records = []
speed_records = []
backtrack_records = []
repeated_records = []
disappear_records = []
early_records = []
offroute_records = []

snapshot_start = df_valid["timestamp"].min()
snapshot_end = df_valid["timestamp"].max()

for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    # 1. Jumps and Gaps
    jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

    # 2. Stuck Vehicles
    stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
    if not stuck.empty:
        stuck_records.append(stuck)

    # 3. Impossible Speeds
    speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
    if not speed.empty:
        speed_records.append(speed)

    # 4. Backtracking
    backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
    if not backtrack.empty:
        backtrack_records.append(backtrack)

    # 5. Repeated Points
    repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
    if not repeat.empty:
        repeated_records.append(repeat)

    # 6. Disappearance
    disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
    if not disappear.empty:
        disappear_records.append(disappear)

    # 7. Early Appearance
    early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
    if not early.empty:
        early_records.append(early)

    # 8. Off-Route
    offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
    if not offroute.empty:
        offroute_records.append(offroute)

# Step 3: Combine to DataFrames
df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# Step 4: Combine all anomalies into a single DataFrame
anomaly_frames = [
    df_anomalies_jumpgap,
    df_anomalies_stuck,
    df_anomalies_speed,
    df_anomalies_backtrack,
    df_anomalies_repeated,
    df_anomalies_disappear,
    df_anomalies_early,
    df_anomalies_offroute
]

anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True)

# Report
print("✅ Unified anomaly count:", len(df_anomalies_full))
display(df_anomalies_full["anomaly_type"].value_counts())


✅ Unified anomaly count: 95248


,count
anomaly_type,
repeated_points,46200
stuck_vehicle,30988
jump_or_gap,9200
off_route,6767
backtracking,1616
impossible_speed,434
disappearance,42
early_appearance,1


In [ ]:
# Ensure shape_id is a column (not index)
shape_route_map = shape_route_map.reset_index()

# Ensure both shape_id columns are string type before merge
df_anomalies_offroute["shape_id"] = df_anomalies_offroute["shape_id"].astype(str)
shape_route_map["shape_id"] = shape_route_map["shape_id"].astype(str)

# Join to enrich anomalies with route info
df_anomalies_offroute = (
    df_anomalies_offroute
    .merge(shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map"))
)

# Patch missing route names
df_anomalies_offroute["route_short_name"] = (
    df_anomalies_offroute["route_short_name"]
    .fillna(df_anomalies_offroute["route_short_name_from_map"])
)

# Clean up
df_anomalies_offroute = df_anomalies_offroute.drop(columns=["route_short_name_from_map"])


In [ ]:
#Sanity Check
missing_routes = df_anomalies_offroute["route_short_name"].isnull().sum()
print(f"Remaining anomalies with missing route_short_name: {missing_routes}")
#If this prints 0, you’ve successfully patched all entries.

if missing_routes > 0:
    print("Sample of anomalies with missing route_short_name:")
    display(df_anomalies_offroute[df_anomalies_offroute["route_short_name"].isnull()].head())


Remaining anomalies with missing route_short_name: 0


In [ ]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 87


#Summary Stats by Route/Anomaly

In [ ]:
# Build trip-to-route lookup from your GTFS static data
trip_to_route_lookup = trips_df[["trip_id", "route_id"]].merge(
    routes_df[["route_id", "route_short_name"]], on="route_id", how="left"
)

# Ensure trip_id types match
df_valid["trip_id"] = df_valid["trip_id"].astype(str)
trip_to_route_lookup["trip_id"] = trip_to_route_lookup["trip_id"].astype(str)

# Drop route_short_name from df_valid if it exists to avoid merge collision
if "route_short_name" in df_valid.columns:
    df_valid = df_valid.drop(columns=["route_short_name"])

# Merge to patch in route_short_name
df_valid = df_valid.merge(trip_to_route_lookup, on="trip_id", how="left")


In [ ]:
#Step 1: Build a lookup table
route_lookup = df_valid[["vehicle_id", "timestamp", "route_short_name"]].copy()
route_lookup["timestamp"] = pd.to_datetime(route_lookup["timestamp"], utc=True)


In [ ]:
#Step 2: Also convert timestamp in anomalies to datetime
df_anomalies_full["timestamp"] = pd.to_datetime(df_anomalies_full["timestamp"], utc=True)


In [ ]:
#Create a backup so we don't have to keep rerunning everything
df_anomalies_full_copy = df_anomalies_full.copy()

# #Rebuild from scratch:

# df_anomalies_full_backup = pd.concat([
    # df_anomalies_jumpgap,
    # df_anomalies_stuck,
    # df_anomalies_speed,
    # df_anomalies_backtrack,
    # df_anomalies_repeated,
    # df_anomalies_disappear,
    # df_anomalies_early,
    # df_anomalies_offroute
# ], ignore_index=True)

In [ ]:
# df_anomalies_full = df_anomalies_full_copy

✅ df_anomalies_full_copy has the correct anomaly types:
repeated_points     46200  
stuck_vehicle       30988  
jump_or_gap          9200  ← LOST  
off_route            6767  
backtracking         1616  
impossible_speed      434  ← LOST  
disappearance          42  ← LOST  
early_appearance        1  ← LOST  
But after the route name merge, your current df_anomalies_full is missing the last four types.


In [ ]:
##After much debugging...
# 🧼 Step 1: Clean merge inputs
valid_rows = df_anomalies_full["timestamp"].notna()
merge_subset = df_anomalies_full.loc[valid_rows].copy()
route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# 🔧 Ensure matching dtypes
merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

# 🔄 Step 2: merge_asof on valid rows only
merge_result = pd.merge_asof(
    merge_subset.sort_values("timestamp"),
    route_lookup_clean.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    tolerance=pd.Timedelta("60s"),
    suffixes=("", "_from_lookup")
)

# 🧩 Step 3: Create mapping from original index
route_name_map = merge_result["route_short_name"].combine_first(
    merge_result.get("route_short_name_from_lookup")
)
route_name_map.index = merge_subset.index

# 🔁 Step 4: Patch back to full dataset
df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(route_name_map)

# ✅ Step 5: Final checks
print("✅ Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
print("✅ Anomaly type breakdown:")
print(df_anomalies_full["anomaly_type"].value_counts(dropna=False))


✅ Remaining missing route names: 9677
✅ Anomaly type breakdown:
anomaly_type
repeated_points     46200
stuck_vehicle       30988
jump_or_gap          9200
off_route            6767
backtracking         1616
impossible_speed      434
disappearance          42
early_appearance        1
Name: count, dtype: int64


/tmp/ipython-input-50-563861858.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  route_name_map = merge_result["route_short_name"].combine_first(


In [ ]:
df_anomalies_full[df_anomalies_full["route_short_name"].isna()].sample(5)


,vehicle_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,snapshot_id,timestamp_collected,...,route_url,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,last_seen,first_seen,route_short_name,distance_from_route_m
40226,359,NaT,2025-07-22 19:17:24.354705+00:00,NaN,979.208002,NaN,NaN,impossible_speed,NaN,NaT,...,NaN,NaN,NaN,NaN,NaT,116.621969,NaT,NaT,NaN,NaN
3610,467,2025-07-23 00:45:39.664712+00:00,2025-07-23 00:46:09.883063+00:00,30.218351,693.363831,False,True,jump_or_gap,NaN,NaT,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,NaN
817,362,2025-07-23 11:37:30.423951+00:00,2025-07-23 11:38:00.645501+00:00,30.221550,555.751984,False,True,jump_or_gap,NaN,NaT,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,NaN
2111,382,2025-07-22 19:58:46.942511+00:00,2025-07-22 19:59:17.190921+00:00,30.248410,700.177794,False,True,jump_or_gap,NaN,NaT,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,NaN
758,361,2025-07-23 13:48:11.665313+00:00,2025-07-23 13:48:42.453363+00:00,30.788050,506.916516,False,True,jump_or_gap,NaN,NaT,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,NaN


##Debugging waypoint 2507251634

In [ ]:
# Treat 'Unknown' as null
df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# Only apply fallback if the lookup column is still present
if "route_short_name_from_lookup" in df_anomalies_full.columns:
    df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(
        df_anomalies_full["route_short_name_from_lookup"]
    )

# Confirm
print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


Remaining missing route names: 9677
route_short_name
16     10000
NaN     9677
66      7894
141     7198
766     7012
Name: count, dtype: int64


In [ ]:
# Confirm null replacement and summary
df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# Summary diagnostics
print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


Remaining missing route names: 9677
route_short_name
16     10000
NaN     9677
66      7894
141     7198
766     7012
Name: count, dtype: int64


In [ ]:
print("Checkpoint anomaly type breakdown:")
print(df_anomalies_full["anomaly_type"].value_counts(dropna=False))

print("Checkpoint anomaly type breakdown (copy):")
print(df_anomalies_full_copy["anomaly_type"].value_counts(dropna=False))

Checkpoint anomaly type breakdown:
anomaly_type
repeated_points     46200
stuck_vehicle       30988
jump_or_gap          9200
off_route            6767
backtracking         1616
impossible_speed      434
disappearance          42
early_appearance        1
Name: count, dtype: int64
Checkpoint anomaly type breakdown (copy):
anomaly_type
repeated_points     46200
stuck_vehicle       30988
jump_or_gap          9200
off_route            6767
backtracking         1616
impossible_speed      434
disappearance          42
early_appearance        1
Name: count, dtype: int64


In [ ]:
print("Total in pivot:", anomaly_by_route_type.sum().sum())
print("Total in df:", df_anomalies_full["anomaly_type"].notna().sum())


Total in pivot: 52967
Total in df: 95248


In [ ]:
# df_temp = df_anomalies_full.copy()
# df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")

# # Ensure all anomaly types are treated as strings before sorting
# full_anomaly_types = df_temp["anomaly_type"].dropna().astype(str).unique()

# anomaly_by_route_type = df_temp.pivot_table(
#     index="route_short_name",
#     columns="anomaly_type",
#     aggfunc="size",
#     fill_value=0
# ).reindex(columns=sorted(full_anomaly_types), fill_value=0).sort_index()

# print("Total in pivot:", anomaly_by_route_type.sum().sum())
# print("Total in df:", df_anomalies_full["anomaly_type"].notna().sum())


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
# df_temp = df_anomalies_full.copy()
# df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")

# # Fully sanitize the anomaly_type list
# full_anomaly_types = (
#     df_temp["anomaly_type"]
#     .dropna()
#     .apply(str)
#     .unique()
# )
# full_anomaly_types = sorted([x for x in full_anomaly_types if isinstance(x, str)])

# anomaly_by_route_type = df_temp.pivot_table(
#     index="route_short_name",
#     columns="anomaly_type",
#     aggfunc="size",
#     fill_value=0
# ).reindex(columns=full_anomaly_types, fill_value=0).sort_index()

# print("Total in pivot:", anomaly_by_route_type.sum().sum())
# print("Total in df:", df_anomalies_full["anomaly_type"].notna().sum())


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
# # Step 1: Fill route_short_name for clean indexing
# df_temp = df_anomalies_full.copy()
# df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")

# # Step 2: Force all anomaly_type entries to string
# df_temp["anomaly_type"] = df_temp["anomaly_type"].astype(str)

# # Step 3: Rebuild complete list of anomaly_type strings
# full_anomaly_types = sorted(df_temp["anomaly_type"].dropna().unique())

# # Step 4: Pivot and reindex to guarantee all anomaly types are included
# anomaly_by_route_type = df_temp.pivot_table(
#     index="route_short_name",
#     columns="anomaly_type",
#     aggfunc="size",
#     fill_value=0
# ).reindex(columns=full_anomaly_types, fill_value=0).sort_index()

# # Step 5: Diagnostic
# print("✅ Total in pivot:", anomaly_by_route_type.sum().sum())
# print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
print("Unique types in anomaly_type:")
print(set(type(val) for val in df_anomalies_full["anomaly_type"].unique()))


Unique types in anomaly_type:
{<class 'str'>}


In [ ]:
# # Copy and clean the dataframe
# df_temp = df_anomalies_full.copy()
# df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")

# # Convert all anomaly_type entries to string and drop NaN
# df_temp["anomaly_type_clean"] = df_temp["anomaly_type"].astype(str)

# # Generate full list of anomaly types as strings
# full_anomaly_types = sorted(df_temp["anomaly_type_clean"].unique())

# # Pivot using cleaned column
# anomaly_by_route_type = df_temp.pivot_table(
#     index="route_short_name",
#     columns="anomaly_type_clean",
#     aggfunc="size",
#     fill_value=0
# ).reindex(columns=full_anomaly_types, fill_value=0).sort_index()

# # Diagnostics
# print("✅ Total in pivot:", anomaly_by_route_type.sum().sum())
# print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
# print("✅ Columns in pivot:", anomaly_by_route_type.columns.tolist())


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
print("Full anomaly types (raw):", df_anomalies_full["anomaly_type"].unique())
print("Full anomaly types (with types):", [(val, type(val)) for val in df_anomalies_full["anomaly_type"].unique()])


Full anomaly types (raw): ['jump_or_gap' 'stuck_vehicle' 'impossible_speed' 'backtracking'
 'repeated_points' 'disappearance' 'early_appearance' 'off_route']
Full anomaly types (with types): [('jump_or_gap', <class 'str'>), ('stuck_vehicle', <class 'str'>), ('impossible_speed', <class 'str'>), ('backtracking', <class 'str'>), ('repeated_points', <class 'str'>), ('disappearance', <class 'str'>), ('early_appearance', <class 'str'>), ('off_route', <class 'str'>)]


In [ ]:
# Step 1: Clean anomaly type column
df_temp = df_anomalies_full.copy()
df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")
df_temp["anomaly_type_clean"] = df_temp["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot = df_temp.pivot_table(
    index="route_short_name",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot = pivot.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot.columns.tolist())


✅ Total in pivot: 95248
✅ Total in df: 95248
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed', 'disappearance', 'early_appearance']


Key Fix Summary (for documentation or future review):
- Problem cause: pivot_table(...).reindex(columns=sorted(...)) introduced TypeError due to sorting mixed types (possibly hidden NaNs or object-type weirdness in internal index representation).

- Confirmed all anomaly_type values were str, but reindexing still failed — indicating a more subtle index-level mismatch.

Solution: Explicitly cast anomaly_type to string and reindex with a predefined list of expected columns.

In [ ]:
pivot

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed,disappearance,early_appearance
route_short_name,,,,,,,,
1,1783,1166,0,246,130,0,0,0
2,761,551,0,91,37,0,0,0
5,2454,1532,0,448,67,0,0,0
8,3117,1830,0,433,54,0,0,0
10,1790,1404,0,368,33,0,0,0
11,1449,1204,0,84,90,0,0,0
16,5417,3716,0,717,150,0,0,0
31,1492,942,0,203,85,0,0,0
36,1285,789,0,163,10,0,0,0


Interestingly, the (Missing) route_short_name has the only instances of jump_or_gap, impossible_speed, disappearance, early_appearance


That’s a critical diagnostic insight — and it likely explains why those anomaly types kept disappearing in earlier pivot tables:

🔍 Root Cause:
By default, pivot_table uses only the combination of index and columns values that exist. If all rows for some anomaly types are associated with a missing route_short_name (NaN), and you drop or exclude these during preprocessing, those types vanish from the pivot.

When you did:

df["route_short_name"] = df["route_short_name"].fillna("(Missing)")

You ensured that:

Those rows were not excluded from the index.

Their anomaly types were included and preserved in the pivot.

✅ Implications:
You must preserve (Missing) rows if you want a complete count across all anomaly types.

If you filter by route, or use .dropna() on route_short_name, anomaly types with only missing routes will be silently excluded.

When you view a pivot per route, types like jump_or_gap, impossible_speed, and disappearance may seem to “disappear” — but they’re actually only found in the (Missing) bucket.

---
To ensure full awareness in reporting or visualizing these anomaly types, do one of the following:

Always include a (Missing) category in route-level pivots.

Add a diagnostic count like:


In [ ]:
print("Anomalies with missing route names by type:")
print(df_anomalies_full[df_anomalies_full["route_short_name"].isna()]["anomaly_type"].value_counts())


Anomalies with missing route names by type:
anomaly_type
jump_or_gap         9200
impossible_speed     434
disappearance         42
early_appearance       1
Name: count, dtype: int64


In [ ]:
# anomaly_by_route_type = df_anomalies_full.pivot_table(
#     index="route_short_name",
#     columns="anomaly_type",
#     aggfunc="size",
#     fill_value=0
# ).sort_index()

# anomaly_by_route_type


In [ ]:
# # Define full list of expected anomaly types
# anomaly_categories = [
#     "jump",
#     "disappearance",
#     "reappearance",
#     "stationary",
#     "impossible_speed",
#     "heading_reversal",
#     "off_route",
#     "repeated_points",
#     "early_appearance",
#     "final_disappearance"
# ]

# # Set anomaly_type as categorical with full categories
# df_anomalies_full["anomaly_type"] = pd.Categorical(
#     df_anomalies_full["anomaly_type"],
#     categories=anomaly_categories,
#     ordered=False
# )

# # Pivot: Route × Anomaly Type count matrix (with 0 fill)
# anomaly_by_route_type = df_anomalies_full.pivot_table(
#     index="route_short_name",
#     columns="anomaly_type",
#     aggfunc="size",
#     fill_value=0
# ).sort_index()

# # Display result
# anomaly_by_route_type


In [ ]:
df_anomalies_full["anomaly_type"].value_counts(dropna=False)


,count
anomaly_type,
repeated_points,46200
stuck_vehicle,30988
jump_or_gap,9200
off_route,6767
backtracking,1616
impossible_speed,434
disappearance,42
early_appearance,1


In [ ]:
# df_anomalies_full[df_anomalies_full["anomaly_type"].isna()].sample(5)


#Plot in Folium

In [ ]:
# #Step 1: Ensure latitude and longitude are floats
# df_valid["latitude"] = pd.to_numeric(df_valid["latitude"], errors="coerce")
# df_valid["longitude"] = pd.to_numeric(df_valid["longitude"], errors="coerce")

# #Step 2: Ensure timestamp_collected is datetime
# df_valid["timestamp_collected"] = pd.to_datetime(df_valid["timestamp_collected"], errors="coerce", utc=True)


In [ ]:
# Step 1: Create a working copy
df_plot_base = df_anomalies_full.copy()

# Step 2: Drop rows with missing coordinates or timestamps
df_plot_base = df_plot_base.dropna(subset=["latitude", "longitude", "timestamp_collected"])

# Step 3: Ensure coordinate and timestamp types are correct
df_plot_base["latitude"] = pd.to_numeric(df_plot_base["latitude"], errors="coerce")
df_plot_base["longitude"] = pd.to_numeric(df_plot_base["longitude"], errors="coerce")
df_plot_base["timestamp_collected"] = pd.to_datetime(df_plot_base["timestamp_collected"], errors="coerce", utc=True)

# Step 4: Optional — Filter for anomalies with known route_short_name
df_plot_base = df_plot_base[df_plot_base["route_short_name"].notna()]

# Step 5: Optional — Save a filtered subset by anomaly type, route, or vehicle for visualization
# You can update these filters dynamically for an interactive tool
selected_anomaly_type = "stuck_vehicle"
selected_route = "66"
# Example: match on anomaly type and route
df_plot_filtered = df_plot_base[
    (df_plot_base["anomaly_type"] == selected_anomaly_type) &
    (df_plot_base["route_short_name"] == selected_route)
]

# Preview result
print(f"🗺️ Plotting subset for anomaly: '{selected_anomaly_type}' on route '{selected_route}'")
print(df_plot_filtered[["vehicle_id", "timestamp_collected", "latitude", "longitude", "anomaly_type"]].head())


🗺️ Plotting subset for anomaly: 'stuck_vehicle' on route '66'
Empty DataFrame
Columns: [vehicle_id, timestamp_collected, latitude, longitude, anomaly_type]
Index: []


/tmp/ipython-input-120-2658071289.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plot_base["latitude"] = pd.to_numeric(df_plot_base["latitude"], errors="coerce")
/tmp/ipython-input-120-2658071289.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plot_base["longitude"] = pd.to_numeric(df_plot_base["longitude"], errors="coerce")
/tmp/ipython-input-120-2658071289.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [ ]:
# # Use the full cleaned dataframe
# df_full = df_valid.copy()

# # Compute diffs for position and time
# df_full["lat_diff"] = df_full.groupby("vehicle_id")["latitude"].diff()
# df_full["lon_diff"] = df_full.groupby("vehicle_id")["longitude"].diff()
# df_full["jump_dist"] = (df_full["lat_diff"]**2 + df_full["lon_diff"]**2)**0.5

# df_full["time_diff"] = df_full.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# # Label events
# df_full["is_jump"] = df_full["jump_dist"] > JUMP_DISTANCE_THRESHOLD
# df_full["is_disappearance"] = df_full["time_diff"] > DISAPPEARANCE_TIME_THRESHOLD

# # Enrich jump rows with previous position
# jumps_df = df_full[df_full["is_jump"]].copy()
# jumps_df["lat_prev"] = df_full.groupby("vehicle_id")["latitude"].shift()
# jumps_df["lon_prev"] = df_full.groupby("vehicle_id")["longitude"].shift()
# jumps_df["timestamp_prev"] = df_full.groupby("vehicle_id")["timestamp_collected"].shift()
# jumps_df["timestamp_curr"] = jumps_df["timestamp_collected"]

# # Filter for vehicles with sufficient jumps
# jump_counts = jumps_df["vehicle_id"].value_counts()
# keep_jumpers = jump_counts[jump_counts >= MIN_JUMP_COUNT_PER_VEHICLE].index
# jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# # Recalculate filtered set for map display
# df_jumpers_only = df_full[df_full["vehicle_id"].isin(keep_jumpers)].copy()

# # Identify disappearance and reappearance points
# disappear_df = df_jumpers_only[df_jumpers_only["is_disappearance"]].copy()
# # Shift the is_disappearance column and fill any resulting NaNs with False
# is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)
# reappear_df = df_jumpers_only[is_reappear].copy()


In [ ]:
# Use full anomaly dataframe
df_full = df_anomalies_full.copy()

# Ensure lat/lon are floats
df_full["latitude"] = pd.to_numeric(df_full["latitude"], errors="coerce")
df_full["longitude"] = pd.to_numeric(df_full["longitude"], errors="coerce")

# Ensure timestamp_collected is datetime
df_full["timestamp_collected"] = pd.to_datetime(df_full["timestamp_collected"], errors="coerce", utc=True)

# Compute diffs for position and time
df_full["lat_diff"] = df_full.groupby("vehicle_id")["latitude"].diff()
df_full["lon_diff"] = df_full.groupby("vehicle_id")["longitude"].diff()
df_full["jump_dist"] = (df_full["lat_diff"]**2 + df_full["lon_diff"]**2)**0.5
df_full["time_diff"] = df_full.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# Label jump and disappearance events using anomaly_params
df_full["is_jump"] = df_full["jump_dist"] > anomaly_params["JUMP_DISTANCE_THRESHOLD"]
df_full["is_disappearance"] = df_full["time_diff"] > anomaly_params["DISAPPEARANCE_TIME_THRESHOLD"]

# Create enriched jump DataFrame
jumps_df = df_full[df_full["is_jump"]].copy()
jumps_df["lat_prev"] = df_full.groupby("vehicle_id")["latitude"].shift()
jumps_df["lon_prev"] = df_full.groupby("vehicle_id")["longitude"].shift()
jumps_df["timestamp_prev"] = df_full.groupby("vehicle_id")["timestamp_collected"].shift()
jumps_df["timestamp_curr"] = jumps_df["timestamp_collected"]

# Filter to vehicles with enough jumps
jump_counts = jumps_df["vehicle_id"].value_counts()
keep_jumpers = jump_counts[jump_counts >= anomaly_params["MIN_JUMP_COUNT_PER_VEHICLE"]].index
jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# Base frame of jumpers for disappearance/reappearance
df_jumpers_only = df_full[df_full["vehicle_id"].isin(keep_jumpers)].copy()

# Identify disappearance and reappearance points
disappear_df = df_jumpers_only[df_jumpers_only["is_disappearance"]].copy()
is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)
reappear_df = df_jumpers_only[is_reappear].copy()

# Summary of Key Changes:
# ✅ Replaces JUMP_DISTANCE_THRESHOLD, DISAPPEARANCE_TIME_THRESHOLD, and MIN_JUMP_COUNT_PER_VEHICLE with their anomaly_params equivalents.
# ✅ Maintains existing structure and logic.
# ✅ Compatible with all downstream Folium plotting layers.


/tmp/ipython-input-121-4130529949.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)


In [ ]:
# #Here is the full code to patch route_short_name into shapes_df, allowing each shape to be associated with its route for proper filtering in Folium:
# # Step 1: Extract shape_id to route_id mapping from trips_df
# shape_to_route = trips_df[["shape_id", "route_id"]].drop_duplicates()

# # Step 2: Map route_id to route_short_name from routes_df
# route_id_to_name = routes_df[["route_id", "route_short_name"]]

# # Step 3: Merge to associate shape_id with route_short_name
# shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")

# # Step 4: Merge back into shapes_df to add route_short_name column
# shapes_df = shapes_df.merge(shape_route_map, on="shape_id", how="left")

# #Adjustments for Route Shape Mapping Block: This block looks solid assuming: trips_df, routes_df, and shapes_df are correctly loaded from your GTFS static files
# #You’ve verified there are no missing joins (e.g., null route_short_name after the final merge). Suggested light addition at the end:
# assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"

# # Step 5: Confirm result
# print("Sample of shapes_df with route_short_name:")
# print(shapes_df[["shape_id", "route_id", "route_short_name"]].drop_duplicates().head())

# #This ensures that each shape_id in your GTFS shapes.txt data now carries the corresponding route_short_name.
# #We can now group route shapes per route like this:
# #for route_name, group in shapes_df.groupby("route_short_name"): ...



In [ ]:
# that traceback confirms the immediate issue: after merging into shapes_df, the column route_short_name is not present, likely due to:

# A failed merge (i.e. no matching shape_id values across shapes_df and shape_route_map).

# The column being named something else (e.g. a merge conflict that renamed it, or a typo).

# An earlier shapes_df overwrite or clean-up step that removed key columns.

print("🔍 Columns in shapes_df:", shapes_df.columns.tolist())

# Check if shape_route_map exists and contains route_short_name
if 'shape_route_map' in globals():
    print("✅ Columns in shape_route_map:", shape_route_map.columns.tolist())
    print("✅ Sample shape_route_map rows:")
    display(shape_route_map.head())
else:
    print("❌ shape_route_map does not exist")

# Check shape_id overlap
if 'shapes_df' in globals():
    print("🧪 shape_id in shapes_df:", shapes_df["shape_id"].nunique())
    print("🧪 shape_id in shape_route_map:", shape_route_map["shape_id"].nunique())
    print("🧪 Overlapping shape_ids:", len(set(shapes_df["shape_id"]).intersection(shape_route_map["shape_id"])))


🔍 Columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id', 'route_short_name']
✅ Columns in shape_route_map: ['shape_id', 'route_id', 'route_short_name']
✅ Sample shape_route_map rows:


,shape_id,route_id,route_short_name
0,17128,4300,1
1,17129,4300,1
2,17130,4301,10
3,17131,4301,10
4,17132,4301,10


🧪 shape_id in shapes_df: 67
🧪 shape_id in shape_route_map: 67
🧪 Overlapping shape_ids: 67


In [ ]:
# assert (shapes_df["route_short_name_x"] == shapes_df["route_short_name_y"]).all(), \
#     "❌ Mismatch between merged route_short_name columns!"


In [ ]:
# # Step 1: Choose which 'route_short_name' to keep
# # In this case, either is fine — they should be identical
# shapes_df["route_short_name"] = shapes_df["route_short_name_x"]

# # Step 2: Drop the extra columns
# shapes_df.drop(columns=[
#     "route_id_x", "route_id_y",
#     "route_short_name_x", "route_short_name_y"
# ], inplace=True)

# # Step 3: Confirm final structure
# print("✅ Cleaned shapes_df columns:", shapes_df.columns.tolist())
# print(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())


In [ ]:
# #Step 1: Extract shape_id to route_id from trips_df
# shape_to_route = trips_df[["shape_id", "route_id"]].drop_duplicates()
# # trips.txt defines which route uses which shape.
# # shape_to_route gives a clean 1:1 or 1:many mapping.

# #Step 2: Map route_id to route_short_name from routes_df
# route_id_to_name = routes_df[["route_id", "route_short_name"]]
# #Brings in human-readable route names like "1", "10", "66".

# #Step 3: Merge to associate shape_id with route_short_name
# shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
# #You now have a table with: shape_id, route_id, and route_short_name

# #Step 4: Merge into shapes_df
# shapes_df = shapes_df.merge(shape_route_map, on="shape_id", how="left")
# #This gave you duplicate route columns: route_id_x, route_id_y, route_short_name_x, route_short_name_y.

# #Step 4.5: Clean up duplicate columns
# shapes_df["route_short_name"] = shapes_df["route_short_name_x"]
# shapes_df.drop(columns=[
#     "route_id_x", "route_id_y",
#     "route_short_name_x", "route_short_name_y"
# ], inplace=True)
# #Confirmed that the two route_short_name columns were identical before dropping.

# #Step 5: Confirm and validate
# assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
# # You verified there were no missing joins.
# # Output looks clean and ready for grouping:
# shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head()



In [ ]:
# #Step 1: Extract shape_id to route_id from trips_df
# shape_to_route = trips_df[["shape_id", "route_id"]].drop_duplicates()
# # trips.txt defines which route uses which shape.
# # shape_to_route gives a clean 1:1 or 1:many mapping.

# #Step 2: Map route_id to route_short_name from routes_df
# route_id_to_name = routes_df[["route_id", "route_short_name"]]
# #Brings in human-readable route names like "1", "10", "66".

# #Step 3: Merge to associate shape_id with route_short_name
# shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
# #You now have a table with: shape_id, route_id, and route_short_name

# #Step 4: Merge into shapes_df
# shapes_df = shapes_df.merge(shape_route_map, on="shape_id", how="left")
# #This gave you duplicate route columns: route_id_x, route_id_y, route_short_name_x, route_short_name_y.

# # Step 4.5 (Safe Cleanup): Use existing column names only
# expected_dupes = ["route_id_x", "route_id_y", "route_short_name_x", "route_short_name_y"]
# existing_dupes = [col for col in expected_dupes if col in shapes_df.columns]

# # If both route_short_name_x and _y exist, reconcile and drop
# if "route_short_name_x" in existing_dupes and "route_short_name_y" in existing_dupes:
#     assert (shapes_df["route_short_name_x"] == shapes_df["route_short_name_y"]).all(), \
#         "❌ Mismatch between merged route_short_name columns!"
#     shapes_df["route_short_name"] = shapes_df["route_short_name_x"]

# # Drop safely
# shapes_df.drop(columns=existing_dupes, inplace=True)

# # Confirm result
# assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
# print("✅ Final columns in shapes_df:", shapes_df.columns.tolist())
# display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())




In [ ]:
#Step 1: Extract shape_id to route_id from trips_df
shape_to_route = trips_df[["shape_id", "route_id"]].drop_duplicates()
# trips.txt defines which route uses which shape.
# shape_to_route gives a clean 1:1 or 1:many mapping.

#Step 2: Map route_id to route_short_name from routes_df
route_id_to_name = routes_df[["route_id", "route_short_name"]]
#Brings in human-readable route names like "1", "10", "66".

#Step 3: Merge to associate shape_id with route_short_name
shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
#You now have a table with: shape_id, route_id, and route_short_name

# Step 4: Drop preexisting route_short_name to avoid merge conflicts
if "route_short_name" in shapes_df.columns:
    shapes_df.drop(columns=["route_short_name"], inplace=True)

# Step 4 (retry): Merge into shapes_df
shapes_df = shapes_df.merge(shape_route_map, on="shape_id", how="left")

# Step 5: Confirm structure
assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
print("✅ Final columns in shapes_df:", shapes_df.columns.tolist())
display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())





✅ Final columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id_x', 'route_id_y', 'route_id', 'route_short_name']


,shape_id,route_short_name
0,17128,1
251,17129,1
441,17130,10
608,17131,10
685,17132,10


In [ ]:
# # --- Step 1: Map shape_id to route_id ---
# shape_to_route = trips_df[["shape_id", "route_id"]].drop_duplicates()

# # Sanity check: Any missing route_id in trips_df?
# if shape_to_route["route_id"].isna().any():
#     print("⚠️ Warning: Some shape_id entries in trips_df are missing route_id!")

# # --- Step 2: Map route_id to route_short_name ---
# route_id_to_name = routes_df[["route_id", "route_short_name"]]

# # Sanity check: Ensure uniqueness of route_id
# assert route_id_to_name["route_id"].is_unique, "❌ route_id should be unique in routes_df"

# # --- Step 3: Merge to associate shape_id with route_short_name ---
# shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")

# # Optional: warn if some mappings failed
# missing_route_names = shape_route_map["route_short_name"].isna().sum()
# if missing_route_names > 0:
#     print(f"⚠️ {missing_route_names} shape_ids could not be mapped to a route_short_name")

# # --- Step 4: Merge route_short_name into shapes_df ---
# shapes_df = shapes_df.merge(shape_route_map, on="shape_id", how="left")

# # --- Step 5: Final validation ---
# assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route_short_name after merge!"

# # --- Step 6: Preview ---
# print("✅ Sample of shapes_df with route_short_name:")
# print(shapes_df[["shape_id", "route_id", "route_short_name"]].drop_duplicates().head())


In [ ]:
# Next Step: Clean up unnecessary suffix columns
# Since the final route_short_name column is already correct, and we don’t need the duplicated *_x and *_y columns, you can safely drop them:
# Clean up leftover duplicate columns
cols_to_drop = [col for col in shapes_df.columns if col.endswith("_x") or col.endswith("_y")]
shapes_df.drop(columns=cols_to_drop, inplace=True)

# Final confirmation
print("✅ Cleaned columns in shapes_df:", shapes_df.columns.tolist())
display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())


✅ Cleaned columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id', 'route_short_name']


,shape_id,route_short_name
0,17128,1
251,17129,1
441,17130,10
608,17131,10
685,17132,10


Summary of Current shapes_df Columns:
- shape_id: Unique shape identifier
- shape_pt_lat, shape_pt_lon: GPS points to trace the route shape
- shape_pt_sequence: Order of points to draw the shape
- shape_dist_traveled: Optional, often used to interpolate positions
- route_id, route_short_name: Now both included — route_short_name is the human-readable one you’ll want for plotting and filtering

In [ ]:
# # --- Create base map ---
# mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# # --- Define color map for route-based layers ---
# route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique())
# cmap = plt.get_cmap("tab20", len(route_names))
# color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# # --- Add anomaly points by (route, anomaly_type) ---
# for (route, anomaly), subset in df_anomalies_full.groupby(["route_short_name", "anomaly_type"]):
#     group = FeatureGroup(name=f"{route} – {anomaly}", show=False)
#     for _, row in subset.iterrows():
#         popup_text = (
#             f"Anomaly: {anomaly}<br>"
#             f"Route: {route}<br>"
#             f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
#             f"Timestamp: {row.get('timestamp', 'N/A')}"
#         )
#         CircleMarker(
#             location=[row["latitude"], row["longitude"]],
#             radius=4,
#             color=color_map.get(route, "black"),
#             fill=True,
#             fill_opacity=0.9,
#             popup=popup_text
#         ).add_to(group)
#     group.add_to(mymap)

# # --- Add jump markers and lines by route ---
# if 'jumps_df' in globals() and not jumps_df.empty:
#     for route, group_df in jumps_df.groupby("route_short_name"):
#         jump_line_group = FeatureGroup(name=f"{route} – Jump Lines", show=False)
#         jump_point_group = FeatureGroup(name=f"{route} – Jump Start/End", show=False)

#         for _, row in group_df.iterrows():
#             start = [row["lat_prev"], row["lon_prev"]]
#             end = [row["latitude"], row["longitude"]]
#             vehicle = row.get("vehicle_id", "N/A")
#             t_prev = row.get("timestamp_prev", "N/A")
#             t_curr = row.get("timestamp_curr", "N/A")

#             PolyLine(
#                 locations=[start, end],
#                 color="orange", weight=2,
#                 tooltip=f"Vehicle {vehicle} jump"
#             ).add_to(jump_line_group)

#             CircleMarker(
#                 location=start, radius=4, color="blue", fill=True,
#                 fill_opacity=0.9,
#                 popup=f"Vehicle {vehicle} START<br>{t_prev}"
#             ).add_to(jump_point_group)

#             CircleMarker(
#                 location=end, radius=4, color="purple", fill=True,
#                 fill_opacity=0.9,
#                 popup=f"Vehicle {vehicle} END<br>{t_curr}"
#             ).add_to(jump_point_group)

#         jump_line_group.add_to(mymap)
#         jump_point_group.add_to(mymap)

# # --- Add disappearances and reappearances by route ---
# if 'disappear_df' in globals() and not disappear_df.empty:
#     for route, group_df in disappear_df.groupby("route_short_name"):
#         disappear_group = FeatureGroup(name=f"{route} – Disappearances", show=False)
#         for _, row in group_df.iterrows():
#             Marker(
#                 location=[row["latitude"], row["longitude"]],
#                 icon=Icon(color="red", icon="times-circle", prefix="fa"),
#                 tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp')}"
#             ).add_to(disappear_group)
#         disappear_group.add_to(mymap)

# if 'reappear_df' in globals() and not reappear_df.empty:
#     for route, group_df in reappear_df.groupby("route_short_name"):
#         reappear_group = FeatureGroup(name=f"{route} – Reappearances", show=False)
#         for _, row in group_df.iterrows():
#             Marker(
#                 location=[row["latitude"], row["longitude"]],
#                 icon=Icon(color="green", icon="check-circle", prefix="fa"),
#                 tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp')}"
#             ).add_to(reappear_group)
#         reappear_group.add_to(mymap)

# # --- Add route shapes as toggleable layers by route ---
# if 'shapes_df' in globals() and not shapes_df.empty:
#     for route, shape_group in shapes_df.groupby("route_short_name"):
#         route_group = FeatureGroup(name=f"{route} – Route Shape", show=False)
#         for shape_id, shape_data in shape_group.groupby("shape_id"):
#             shape_data = shape_data.sort_values("shape_pt_sequence")
#             latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
#             PolyLine(
#                 locations=latlons,
#                 color=color_map.get(route, "gray"),
#                 weight=2,
#                 opacity=0.6,
#                 popup=f"Route {route} | Shape {shape_id}"
#             ).add_to(route_group)
#         route_group.add_to(mymap)

# # --- Add layer controls ---
# LayerControl(collapsed=False).add_to(mymap)

# # --- Display map ---
# mymap


In [ ]:
# --- Create base map ---
mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# --- Define color map for routes ---
route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique())
cmap = plt.get_cmap("tab20", len(route_names))
color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# --- Plot anomaly markers by route and type ---
for (route, anomaly), subset in df_anomalies_full.groupby(["route_short_name", "anomaly_type"]):
    group = FeatureGroup(name=f"{route} – {anomaly}", show=False)
    for _, row in subset.iterrows():
        if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
            popup = (
                f"Anomaly: {anomaly}<br>"
                f"Route: {route}<br>"
                f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
                f"Timestamp: {row.get('timestamp_collected', 'N/A')}"
            )
            CircleMarker(
                location=[row["latitude"], row["longitude"]],
                radius=4,
                color=color_map.get(route, "black"),
                fill=True,
                fill_opacity=0.9,
                popup=popup
            ).add_to(group)
    group.add_to(mymap)

# --- Plot jump lines and markers ---
if 'jumps_df' in globals() and not jumps_df.empty:
    for route, group_df in jumps_df.groupby("route_short_name"):
        jump_line_group = FeatureGroup(name=f"{route} – Jump Lines", show=False)
        jump_point_group = FeatureGroup(name=f"{route} – Jump Start/End", show=False)
        for _, row in group_df.iterrows():
            if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
                start = [row["lat_prev"], row["lon_prev"]]
                end = [row["latitude"], row["longitude"]]
                vehicle = row.get("vehicle_id", "N/A")
                t_prev = row.get("timestamp_prev", "N/A")
                t_curr = row.get("timestamp_curr", "N/A")

                PolyLine([start, end], color="orange", weight=2,
                         tooltip=f"Vehicle {vehicle} jump").add_to(jump_line_group)

                CircleMarker(location=start, radius=4, color="blue", fill=True,
                             fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(jump_point_group)
                CircleMarker(location=end, radius=4, color="purple", fill=True,
                             fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(jump_point_group)
        jump_line_group.add_to(mymap)
        jump_point_group.add_to(mymap)

# --- Plot disappearances ---
if 'disappear_df' in globals() and not disappear_df.empty:
    for route, group_df in disappear_df.groupby("route_short_name"):
        disappear_group = FeatureGroup(name=f"{route} – Disappearances", show=False)
        for _, row in group_df.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                Marker(
                    location=[row["latitude"], row["longitude"]],
                    icon=Icon(color="red", icon="times-circle", prefix="fa"),
                    tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
                ).add_to(disappear_group)
        disappear_group.add_to(mymap)

# --- Plot reappearances ---
if 'reappear_df' in globals() and not reappear_df.empty:
    for route, group_df in reappear_df.groupby("route_short_name"):
        reappear_group = FeatureGroup(name=f"{route} – Reappearances", show=False)
        for _, row in group_df.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                Marker(
                    location=[row["latitude"], row["longitude"]],
                    icon=Icon(color="green", icon="check-circle", prefix="fa"),
                    tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
                ).add_to(reappear_group)
        reappear_group.add_to(mymap)

# --- Plot route shapes ---
if 'shapes_df' in globals() and not shapes_df.empty:
    for route, group in shapes_df.groupby("route_short_name"):
        route_group = FeatureGroup(name=f"{route} – Route Shape", show=False)
        for shape_id, shape_data in group.groupby("shape_id"):
            shape_data = shape_data.sort_values("shape_pt_sequence")
            latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
            PolyLine(
                locations=latlons,
                color=color_map.get(route, "gray"),
                weight=2,
                opacity=0.6,
                popup=f"Route {route} | Shape {shape_id}"
            ).add_to(route_group)
        route_group.add_to(mymap)

# --- Add layer controls ---
LayerControl(collapsed=False).add_to(mymap)

# --- Display the map ---
mymap